In [ ]:
#install libraries
import gdal
import ogr
import math
from sklearn import metrics
from sklearn import svm
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error

In [ ]:
#Function definition
def rasterizeVector(path_to_vector, cols, rows, geo_transform, projection, n_class, raster):
    lblRaster = np.zeros((rows, cols))
    inputDS = ogr.Open(path_to_vector)
    driver = gdal.GetDriverByName('MEM')
    # Define spatial reference
    for j in range(n_class):
        shpLayer = inputDS.GetLayer(0)
        class_id = j+1
        rasterDS = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
        rasterDS.SetGeoTransform(geo_transform)
        rasterDS.SetProjection(projection)
        shpLayer.SetAttributeFilter("class_id = " + str(class_id))
        bnd = rasterDS.GetRasterBand(1)
        bnd.FlushCache()
        gdal.RasterizeLayer(rasterDS, [1], shpLayer, burn_values=[class_id])
        arr = bnd.ReadAsArray()
        lblRaster += arr
        rasterDS = None
    save_raster = gdal.GetDriverByName('GTiff').Create(raster, cols, rows, 1, gdal.GDT_UInt16)
    sband = save_raster.GetRasterBand(1)
    sband.WriteArray(lblRaster)
    sband.FlushCache()
    return lblRaster


def make_binary_labels(labelraster):
    labels = np.unique(labelraster)
    n = labels.size
    binary_label_dump = list()
    buffer_array = np.zeros_like(labelraster)
    for l in labels:
        np.place(buffer_array, labelraster==2, l)
        binary_label_dump.append(buffer_array)
        buffer_array = np.zeros_like(labelraster)
    return binary_label_dump


def createGeotiff(outRaster, data, geo_transform, projection, dtyp, bcount=1):
    # Create a GeoTIFF file with the given data
    driver = gdal.GetDriverByName('GTiff')
    rows, cols, _ = data.shape
    rasterDS = driver.Create(outRaster, cols, rows, bcount, dtyp)
    rasterDS.SetGeoTransform(geo_transform)
    rasterDS.SetProjection(projection)
    for i in range(bcount):
        band = rasterDS.GetRasterBand(i+1)
        band.WriteArray(data[:, :, i])
        band.FlushCache()
    return 0

def check_accuracy(actual_labels, predicted_labels, label_count):
    error_matrix = np.zeros((label_count, label_count))
    for actual, predicted in zip(actual_labels, predicted_labels):
        #print(type(actual), type(predicted))
        error_matrix[int(actual) - 1][int(predicted) - 1] += 1 
    return error_matrix

In [ ]:
#input files (give the path for input image, training data & testing data shapefile)
inpRaster = r"/Path to image/image.tif" 
shapefile = r"/Path to training data shapefile/ground_data_train.shp"   #training data shapefile
shapefile_test = r"/Path to testing data shapefile/ground_data_test.shp"   #testing data shapefile


In [ ]:
#set output path
outRaster = r"/path to output location/RandomForest.tif"
out_prob = r"/path to output location/Probability_Map.tif"
rasterized_shp = r"/path to output location/Rasterized.tif"
rasterized_shp_test = r"/path to output location/Rasterized_test.tif"


In [ ]:
# Open raster dataset
rasterDS = gdal.Open(inpRaster, gdal.GA_ReadOnly)
# Get spatial reference
geo_transform = rasterDS.GetGeoTransform()
projection = rasterDS.GetProjectionRef()

# Extract band's data and transform into a numpy array
bandsData = []
for b in range(rasterDS.RasterCount):
    band = rasterDS.GetRasterBand(b+1)
    band_arr = band.ReadAsArray()
    bandsData.append(band_arr)
bandsData = np.dstack(bandsData)
cols, rows, noBands = bandsData.shape

In [1]:
# Rasterize all the vectors in the given directory into a single labelled raster
lblRaster = rasterizeVector(shapefile, rows, cols, geo_transform, projection, n_class=6, raster=rasterized_shp) # change n_class: it is number of classes
lblRaster_test = rasterizeVector(shapefile_test, rows, cols, geo_transform, projection, n_class=6, raster=rasterized_shp_test) # change n_class

print('Vectors Rasterized to Raster!')

In [ ]:
# Prepare training data (set of pixels used for training) and labels
isTrain = np.nonzero(lblRaster)
isTest = np.nonzero(lblRaster_test)
trainingLabels = lblRaster[isTrain]
testingLabels = lblRaster_test[isTest]
# print(np.unique(trainingLabels))
trainingData = bandsData[isTrain]
testingData = bandsData[isTest]

In [2]:
# Train a Random Forest classifier
classifier = RandomForestClassifier(n_jobs=4, n_estimators=500, criterion='gini', oob_score= True, max_features= 19)
classifier.fit(trainingData, trainingLabels)


In [ ]:
# Feature importance
importances = classifier.feature_importances_
std = np.std([tree.feature_importances_ for tree in classifier.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

def importanceFunction(train_data, train_labels, test_data, test_labels, old_acc=0, new_acc=0):
    originalFeatures = len(range(train_data.shape[1]))
    reducedFeatures = int(originalFeatures*0.8)
    print('original features:', originalFeatures)
    classifier.fit(train_data,train_labels)
    old_acc=new_acc
    new_acc = classifier.score(test_data, test_labels)
    print('old accuracy:', old_acc)
    print('new accuracy:', new_acc)
    print('reduced features:', reducedFeatures)
    
    importances = classifier.feature_importances_
    indices = np.argsort(importances)[::-1]
    thld = importances[indices[reducedFeatures - 1]]
    if new_acc > old_acc:
        print('Since new>old, reducing last 20% bands')
        sfm =  SelectFromModel(classifier, threshold = thld, prefit=True)
        train_new = sfm.transform(train_data)
        test_new = sfm.transform(test_data)
        result = importanceFunction(train_new, train_labels, test_new, test_labels, old_acc, new_acc)
    else:
        print('Since old>new, previous iteration gave best accuracy with %d bands' % originalFeatures)
        result = reducedFeatures
    return result;


originalFeatures = len(range(trainingData.shape[1]))
reducedFeatures = int(originalFeatures*0.8)
thld = importances[indices[reducedFeatures - 1]]
old_accuracy = classifier.score(testingData, testingLabels)
print ('reduced features:', reducedFeatures)
new_accuracy = 0.0

importance_result = importanceFunction(trainingData, trainingLabels, testingData, testingLabels);



In [ ]:
#parameter tuning
csv_rf_outputs = list()
csv_rf_outputs.append(['no_tree', 'OOB Score', 'Overall Accuracy', 'Kappa value'])
for no_tree in np.arange(50, 1000, 25):
    classifier =  RandomForestClassifier(n_estimators=no_tree, criterion='gini', n_jobs=4, bootstrap=True, oob_score=True, max_features=19)
    classifier.fit(trainingData, trainingLabels)
    predicted_labels_test = classifier.predict(testingData)
    csv_rf_outputs.append([no_tree, classifier.oob_score_, classifier.score(testingData, testingLabels), cohen_kappa_score(testingLabels, predicted_labels_test)])
    print("no_tree: ",no_tree,"OOB Score: ", classifier.oob_score_,"Overall Accuracy",classifier.score(testingData, testingLabels),"Kappa value:",cohen_kappa_score(testingLabels, predicted_labels_test))
df_out = pd.DataFrame(csv_rf_outputs)
df_out.to_csv("/output location/RandomForest_Ntree.csv")  #set output path

In [ ]:
# Predict class label of unknown pixels
noSamples = rows*cols
flat_pixels = bandsData.reshape((noSamples, noBands))
result = classifier.predict(flat_pixels)
p_vals = classifier.predict_proba(flat_pixels)


In [ ]:
#classification accuracy

print("OOB Score: ", classifier.oob_score_)
predicted_labels = classifier.predict(trainingData)
lbl_cnt = (np.unique(trainingLabels)).size
df = pd.DataFrame(check_accuracy(trainingLabels, predicted_labels, 6))
df.to_csv(r"/output path/CM.csv", index=False)    #set output path

score_oa = classifier.score(trainingData, trainingLabels)
print ('training set OA:', score_oa)
score_oa_test = classifier.score(testingData, testingLabels)
print ('testing set OA:', score_oa_test)

predicted_labels_test = classifier.predict(testingData)
test_lbl_cnt = (np.unique(testingLabels)).size

print('Testing Labels: ',np.unique(testingLabels))
print('Predicted Labels: ',np.unique(predicted_labels_test))

df_test = pd.DataFrame(check_accuracy(testingLabels, predicted_labels_test, 6))
df_test.to_csv('/output path/CM_test.csv', index=False)      #set output path

print('Confusion Matrices Created!')

###kappa value=======
kappa_score = cohen_kappa_score(trainingLabels, predicted_labels)
print ('kappa value training: ', kappa_score)
kappa_score_test = cohen_kappa_score(testingLabels, predicted_labels_test)
print ('kappa value testing: ', kappa_score_test)

rmse_test = math.sqrt(mean_squared_error(testingLabels, predicted_labels_test))
print ("rmse error test: ", rmse_test)

In [ ]:
# Create a GeoTIFF file with the given data
b_count = p_vals.shape[1]

classification = result.reshape((cols, rows, 1))
prob_arr = p_vals.reshape((cols, rows, b_count))

createGeotiff(outRaster, classification, geo_transform, projection, gdal.GDT_UInt16)
createGeotiff(out_prob, prob_arr, geo_transform, projection, gdal.GDT_Float32, b_count)

print('Classified Tiff Image created!')

In [ ]:
#visualise the classified output
img = plt.imread(r"/path to the classified image/RandomForest.tif")    #set input path for the generated classified image
plt.imshow(img)
plt.show()